In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>")) # makes the notebook fill the whole window

import numpy as np
import pandas as pd
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import datetime

# from bokeh.io import output_file, show
# from bokeh.models import ColumnDataSource, GMapOptions
# from bokeh.plotting import gmap

from PIL import Image, ImageDraw
import os
import shutil

from mpl_toolkits.basemap import Basemap

%matplotlib inline

#Turn off interactive plotting for pyplot
plt.ioff()

In [49]:
df = pd.read_csv('./Irma Storm centered/ATL_17_11_Irma_Reduced_Trackfile.txt',header=None,names=["Year","Month","Day","Hour","Lat","Long","Min_Pressure","Max_Winds","Unused"],low_memory=False,sep='\t')
df = df.drop("Unused",axis=1)
ln = pd.read_csv('./Irma Storm centered/ATL_17_11_Irma_WWLLN_Locations.txt',header=None,names=["Year","Month","Day","Hour","Min","Sec","Lat","Long","Dist_East_West","Dist_North_South"],low_memory=False,sep=' ')

In [60]:
#Graph the center of the storm and lightning
for x in range(len(df['Month'].unique())):
    month = df['Month'].unique()[x]
    for y in range(len(df[df['Month'] == df['Month'].unique()[x]]['Day'].unique())):
        day = df[df['Month'] == df['Month'].unique()[x]]['Day'].unique()[y]
        for z in range(len(df[df['Day'] == df[df['Month'] == df['Month'].unique()[x]]['Day'].unique()[y]]['Hour'].unique())):
            hour = df[df['Day'] == df[df['Month'] == df['Month'].unique()[x]]['Day'].unique()[y]]['Hour'].unique()[z]
            for w in range(0,60,30):
                minute = w
                #The above gets the month, day, hour, minute (before 30 minutes or after 30 minutes)
                #Make big map
                fig = plt.figure(figsize=(30,15))
                m = Basemap(llcrnrlon=minLong_-10, llcrnrlat=minLat_-10,urcrnrlon=maxLong_+10,urcrnrlat=maxLat_+10,lon_0=0,lat_0=0)
                m.drawmapboundary(fill_color='#A6CAE0', linewidth=0)
                m.fillcontinents(color='grey', alpha=0.7, lake_color='grey')
                m.drawcoastlines(linewidth=0.1, color="white")
                #Plot ticks for lat/long
                plt.xticks(np.arange(minLong_-10,maxLong_+10,step=5))
                plt.yticks(np.arange(minLat_-10,maxLat_+10,step=5))
                #Title the map 2017:month:day::hour:minute
                plt.title("2017:" + str(month) + ":" + str(day) + "::" + str(hour) + ":" + str(minute))
                #Plot the center of Irma
                m.plot(df[(df['Month'] == month) & (df['Day'] == day) & (df['Hour'] == hour)]['Long'], df[(df['Month'] == month) & (df['Day'] == day) & (df['Hour'] == hour)]['Lat'], linestyle='none', marker="o", markersize=30, alpha=1, c="red", markeredgecolor="black", markeredgewidth=1)
                #Plot all of the lightning that appears on that month/day/hour/minute section
                m.plot(ln[(ln['Month'] == month) & (ln['Day'] == day) & (ln['Hour'] == hour) & (ln['Min'] >= minute) & (ln['Min'] <= minute + 30)]['Long'],ln[(ln['Month'] == month) & (ln['Day'] == day) & (ln['Hour'] == hour) & (ln['Min'] >= minute) & (ln['Min'] <= minute + 30)]['Lat'], linestyle='none', marker="X", markersize=8, alpha=.7, c="yellow", markeredgecolor="black", markeredgewidth=1)
                #Save and close the figure
                plt.savefig("./data/Irma/" + "2017_" + str(month) + "_" + str(day) + "_" + str(hour) + "_" + str(minute),bbox_inches='tight')
                plt.close(fig)